# Capítulo 10 - Casos de Uso e Melhores Práticas

## 🎓 Conclusão do Curso

Chegamos ao final da nossa jornada **DuckDB + Iceberg**.
Começamos configurando um ambiente local, aprendemos a ler e escrever dados, exploramos features avançadas como **Time Travel** e **Schema Evolution**, e finalmente migramos para um ambiente de nuvem simulado com **MinIO**.

Neste último capítulo, vamos resumir as **Melhores Práticas** e discutir **Arquitetura**.

## 🏆 Melhores Práticas

### 1. Particionamento (Hidden Partitioning)
Evite particionar por colunas de alta cardinalidade (ex: `user_id`, `timestamp_preciso`).
Prefira as transformações do Iceberg:
*   `days(timestamp)`
*   `months(timestamp)`
*   `bucket(coluna, N)` -> Ótimo para distribuir dados uniformemente.

A grande vantagem do Iceberg é que **você consulta a coluna original**, e ele descobre a partição sozinho (Hidden Partitioning).

### 2. Manutenção (Compaction & Cleaning)
O Iceberg resolve o problema dos arquivos pequenos ("small files problem") gerados por streaming, mas cria um débito de manutenção.
*   Agende jobs periódicos de `rewrite_data_files` (Compactação).
*   Configure `expire_snapshots` para não guardar histórico infinito (custo de storage).

### 3. Catalog Choice
Para produção em AWS: Use **AWS Glue Catalog**.
Para produção Agnóstica/On-Premise: Use **REST Catalog** (Project Nessie, Gravitino ou Tabular).
Evite usar FileSystem direto (Hadoop/S3FileIO sem catálogo robusto) em ambientes com concorrência massiva.

## 🏗️ Casos de Uso Comuns

### CDC (Change Data Capture)
Iceberg suporta `MERGE INTO`, permitindo aplicar upserts (inserts + updates + deletes) vindos de bancos transacionais (Postgres/MySQL) via Debezium/Kafka.
Isso cria um **Lakehouse Transactional**.

### LGPD / GDPR (Right to be Forgotten)
Diferente de Data Lakes tradicionais (onde deletar um registro num arquivo CSV/Parquet exige reescrever o arquivo inteiro manualmente), o Iceberg permite deletar linhas específicas.
Com `expire_snapshots`, você garante que a versão antiga com o dado sensível foi de fato eliminada fisicamente.

## 🏁 Exemplo Final: Conectando Tudo

Vamos fazer uma leitura final no nosso ambiente **S3 (MinIO)** para confirmar que nossa arquitetura "Cloud" está estável.

In [ ]:
import pyiceberg
from pyiceberg.catalog.sql import SqlCatalog
import s3fs
import pandas as pd
import warnings

# Suprimir avisos de SSL inseguro (comum em dev local)
warnings.filterwarnings('ignore')

# Configuração Cloud (Simulada)
S3_ENDPOINT = "http://localhost:9000"
S3_ACCESS_KEY = "admin"
S3_SECRET_KEY = "password"
BUCKET_NAME = "warehouse"

# Conectando ao Catálogo S3 (criado no Cap 09)
try:
    catalog = SqlCatalog(
        "minio_catalog",
        **{
            "uri": "sqlite:///iceberg_warehouse/s3_catalog.db",
            "warehouse": f"s3://{BUCKET_NAME}/iceberg",
            "s3.endpoint": S3_ENDPOINT,
            "s3.access-key-id": S3_ACCESS_KEY,
            "s3.secret-access-key": S3_SECRET_KEY,
        }
    )
    
    # Carregar tabela
    tbl = catalog.load_table("s3_sales_db.cloud_sales")
    print(f"✅ Conexão estabelecida com sucesso!")
    print(f"📍 Tabela: {tbl.location()}")
    print(f"🔢 Snapshots Atuais: {len(tbl.snapshots())}")
    
    # Leitura Final
    df = tbl.scan().to_pandas()
    print("\n📊 Dados no Lakehouse:")
    display(df.head())
    
except Exception as e:
    print(f"⚠️ Erro ao conectar (Execute o Cap 09 primeiro): {e}")

print("\n🚀 Parabéns! Você concluiu a trilha DuckDB + Iceberg.")